# Usando a ResNet50 para treinar sem data augmentation e sem windowing

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.3 MB/s 


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Ajustando o DS

## Baixando o DS

In [ ]:
!kaggle datasets download -d pedroamaro/smaller-rsna-ds-train-test

100% 7.25G/7.25G [01:40<00:00, 109MB/s] 
100% 7.25G/7.25G [01:40<00:00, 77.8MB/s]


In [ ]:
!unzip /content/smaller-rsna-ds-train-test.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/smaller-rsna-ds-train-test.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_train/no | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_train/yes | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


## Salvando como JPG e aplicando full windowing

In [ ]:
!mkdir small_rsna_ds_as_jpg
!mkdir /content/small_rsna_ds_as_jpg/no
!mkdir /content/small_rsna_ds_as_jpg/yes

!mkdir test_as_jpg
!mkdir test_as_jpg/no
!mkdir test_as_jpg/yes

In [ ]:
def save_W_as_jpg(path, id, dest_folder):

  dcm = pydicom.dcmread(path)

  im = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

### Para o conjunto Train

In [ ]:
train_path = "/content/small_rsna_ds_train/yes/"
folder = os.listdir("/content/small_rsna_ds_train/yes/")
dest_folder = "/content/small_rsna_ds_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
train_path = "/content/small_rsna_ds_train/no/"
folder = os.listdir("/content/small_rsna_ds_train/no/")
dest_folder = "/content/small_rsna_ds_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_as_jpg/yes/ | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


### Para o conjunto Test

In [ ]:
train_path = "/content/small_rsna_ds_test/yes/"
folder = os.listdir("/content/small_rsna_ds_test/yes/")
dest_folder = "/content/test_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

In [ ]:
train_path = "/content/small_rsna_ds_test/no/"
folder = os.listdir("/content/small_rsna_ds_test/no/")
dest_folder = "/content/test_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [ ]:
print('Numero de imagens: NO')
!ls /content/test_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/test_as_jpg/yes/ | wc -l

Numero de imagens: NO
10022
Numero de imagens: YES
1643


# Carregando o dataset

In [ ]:
train_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="training",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
     )

Found 23334 files belonging to 2 classes.
Using 15556 files for training.


In [ ]:
val_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="validation",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
)

Found 23334 files belonging to 2 classes.
Using 7778 files for validation.


In [ ]:
test_data = image_dataset_from_directory(
    "/content/test_as_jpg",
     labels='inferred',
     label_mode="int",
     batch_size = 64,
     color_mode = "rgb"
     )

Found 11665 files belonging to 2 classes.


In [ ]:
class_names = train_data.class_names

## Normalizando e Pegando o tamanho de entrada

In [ ]:
normalization_layer = Rescaling(1./255)

normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))

normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(normalized_val_data))

normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
test_image_batch, test_labels_batch = next(iter(normalized_test_data))

# ResNet

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=20, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/20
244/244 [==============================] - 247s 916ms/step - loss: 0.5720 - auc: 0.7678 - accuracy: 0.6565 - val_loss: 0.4807 - val_auc: 0.3493 - val_accuracy: 0.8581
Epoch 2/20
244/244 [==============================] - 229s 937ms/step - loss: 0.4556 - auc: 0.8604 - accuracy: 0.7383 - val_loss: 0.6875 - val_auc: 0.6634 - val_accuracy: 0.5744
Epoch 3/20
244/244 [==============================] - 230s 941ms/step - loss: 0.3597 - auc: 0.9194 - accuracy: 0.8217 - val_loss: 1.6755 - val_auc: 0.7017 - val_accuracy: 0.3089
Epoch 4/20
244/244 [==============================] - 230s 940ms/step - loss: 0.2551 - auc: 0.9622 - accuracy: 0.8899 - val_loss: 2.5129 - val_auc: 0.7532 - val_accuracy: 0.5149
Epoch 5/20
244/244 [==============================] - 230s 940ms/step - loss: 0.1687 - auc: 0.9844 - accuracy: 0.9287 - val_loss: 2.4156 - val_auc: 0.7752 - val_accuracy: 0.6216
Epoch 6/20
244/244 [==============================] - 231s 946ms/step - loss: 0.0793 - auc: 0.9971 - accuracy:

In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 51s 274ms/step - loss: 0.5784 - auc: 0.8448 - accuracy: 0.8556
[0.5784299969673157, 0.8448062539100647, 0.8555507659912109]
